In [1]:
import os
import pandas as pd
import random
import time
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.callbacks import Callback, EarlyStopping
from sklearn.model_selection import train_test_split  
from tensorflow.keras.applications import MobileNetV2 


In [2]:


# Verify GPU availability
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

# Define the dataset path
dataset_path = 'Food_5K'

# Check and print the existing subdirectories
subdirectories = []
for subset in ['training', 'validation', 'evaluation']:
    subset_path = os.path.join(dataset_path, subset)
    if os.path.exists(subset_path):
        subdirectories.append(subset_path)
    else:
        print(f"Directory {subset_path} does not exist.")

# List all image files in training, validation, and evaluation directories including food and non-food subdirectories
image_files = []

for subset in ['training', 'validation', 'evaluation']:
    subset_path = os.path.join(dataset_path, subset)
    for category in ['food']:
        category_path = os.path.join(subset_path, category)
        if not os.path.exists(category_path):
            print(f"Directory {category_path} does not exist.")
            continue
        for filename in os.listdir(category_path):
            if filename.endswith('.jpg'):
                image_files.append(os.path.join(subset, category, filename))

# Check if we have listed the files correctly
print(f"Total images found: {len(image_files)}")
print("First 10 image files:")
print(image_files[:10])

# Example function to assign random calorie values between 50 and 500
def assign_calories(image_file):
    return random.randint(50, 500)

# Create a list of dictionaries with filenames and calorie values
calorie_data = [{'filename': file, 'calorie': assign_calories(file)} for file in image_files]

# Display the first few entries
print("First 10 entries in calorie_data:")
print(calorie_data[:10])

# Convert the list of dictionaries to a DataFrame
calorie_df = pd.DataFrame(calorie_data)

# Save the DataFrame to a CSV file
calorie_df.to_csv('calorie_info.csv', index=False)

# Display the first few rows of the DataFrame to verify
print("First few rows of calorie_df:")
print(calorie_df.head())




Num GPUs Available:  0
Total images found: 2500
First 10 image files:
['training\\food\\0.jpg', 'training\\food\\1.jpg', 'training\\food\\10.jpg', 'training\\food\\100.jpg', 'training\\food\\1000.jpg', 'training\\food\\1001.jpg', 'training\\food\\1002.jpg', 'training\\food\\1003.jpg', 'training\\food\\1004.jpg', 'training\\food\\1005.jpg']
First 10 entries in calorie_data:
[{'filename': 'training\\food\\0.jpg', 'calorie': 229}, {'filename': 'training\\food\\1.jpg', 'calorie': 361}, {'filename': 'training\\food\\10.jpg', 'calorie': 92}, {'filename': 'training\\food\\100.jpg', 'calorie': 52}, {'filename': 'training\\food\\1000.jpg', 'calorie': 473}, {'filename': 'training\\food\\1001.jpg', 'calorie': 109}, {'filename': 'training\\food\\1002.jpg', 'calorie': 59}, {'filename': 'training\\food\\1003.jpg', 'calorie': 477}, {'filename': 'training\\food\\1004.jpg', 'calorie': 246}, {'filename': 'training\\food\\1005.jpg', 'calorie': 75}]
First few rows of calorie_df:
                 filename 

In [3]:
# ImageDataGenerator for data augmentation
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
validation_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Load pre-trained MobileNetV2 model
base_model = MobileNetV2(include_top=False, input_shape=(224, 224, 3), weights='imagenet')
base_model.trainable = False


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [4]:

# Define the model
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1)  # Regression output for calorie estimation
])

In [5]:

# Compile the model
model.compile(optimizer=optimizers.Adam(),
              loss='mean_squared_error',
              metrics=['mae'])

# Custom callback to measure epoch duration and estimate remaining time
class TimeHistory(Callback):
    def on_train_begin(self, logs=None):
        self.times = []
        self.epoch_start_time = None

    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_start_time = time.time()

    def on_epoch_end(self, epoch, logs=None):
        epoch_duration = time.time() - self.epoch_start_time
        self.times.append(epoch_duration)
        average_epoch_time = sum(self.times) / len(self.times)
        epochs_remaining = self.params['epochs'] - (epoch + 1)
        estimated_time_remaining = epochs_remaining * average_epoch_time
        print(f"Epoch {epoch + 1}/{self.params['epochs']} - Estimated time remaining: {estimated_time_remaining / 60:.2f} minutes")

time_callback = TimeHistory()

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


In [6]:

# Training data generator
training_generator = train_datagen.flow_from_dataframe(
    dataframe=calorie_df,
    directory=dataset_path,
    x_col='filename',
    y_col='calorie',
    target_size=(224, 224),
    batch_size=16,  # Reduced batch size
    class_mode='raw',
    subset='training'
)

# Validation data generator
validation_generator = validation_datagen.flow_from_dataframe(
    dataframe=calorie_df,
    directory=dataset_path,
    x_col='filename',
    y_col='calorie',
    target_size=(224, 224),
    batch_size=16,  # Reduced batch size
    class_mode='raw',
    subset='validation'
)

Found 2000 validated image filenames.
Found 500 validated image filenames.


In [7]:

# Fit the model
history = model.fit(
    training_generator,
    validation_data=validation_generator,
    epochs=1,
    callbacks=[early_stopping, time_callback]
)


C:\Users\91703\anaconda3\anaconda\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 626ms/step - loss: 37417.0312 - mae: 155.5754Epoch 1/1 - Estimated time remaining: 0.00 minutes
125/125 ━━━━━━━━━━━━━━━━━━━━ 116s 822ms/step - loss: 37314.9062 - mae: 155.3613 - val_loss: 20252.6328 - val_mae: 121.2958


In [8]:

# Save the model
model.save('food_calorie_model.h5')


In [12]:

# Predict and print name and caloric value
def predict_and_print(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
    img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0
    img_array = tf.expand_dims(img_array, 0)  # Create batch axis

    prediction = model.predict(img_array)
    predicted_calorie = prediction[0][0]
    print(f"Image: {image_path}, Predicted Calorie: {predicted_calorie}")

# Example usage
predict_and_print('pizza.jpg')  # Update with the path to an actual image


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
Image: pizza.jpg, Predicted Calorie: 176.27816772460938
